In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

1) Se cuenta con un RDD con información sobre patentamientos
de autos con la siguiente información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto.

Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2019. (***) (15 pts)

In [10]:
autos = [('abc 123', 'marca1', 'modelo1', 'version1', 'auto', 'Bs As', '20190401'),
        ('abc 124', 'marca1', 'modelo1', 'version2', 'auto', 'Bs As', '20190401'),
        ('abc 125', 'marca7', 'modelo7', 'version3', 'moto', 'Bs As', '20190401'),
        ('abc 126', 'marca4', 'modelo4', 'version4', 'auto', 'Cordoba', '20190401'),
        ('abc 127', 'marca5', 'modelo5', 'version5', 'camion', 'Bs As', '20190401'),
        ('abc 128', 'marca1', 'modelo1', 'version6', 'auto', 'Bs As', '20190401'),
        ('abc 129', 'marca7', 'modelo7', 'version7', 'moto', 'Bs As', '20190401'),
        ('abc 133', 'marca1', 'modelo1', 'version8', 'auto', 'Bs As', '20190401'),
        ('abc 143', 'marca9', 'modelo9', 'version9', 'auto', 'Bs As', '20190401'),
        ('abc 153', 'marca9', 'modelo9', 'version10', 'auto', 'Bs As', '20190401'),
        ('abc 163', 'marca11', 'modelo11', 'version11', 'moto', 'Bs As', '20190401'),
        ('abc 173', 'marca12', 'modelo12', 'version12', 'auto', 'Bs As', '20190301'),
        ('abc 183', 'marca12', 'modelo12', 'version13', 'auto', 'Bs As', '20190501')]

In [11]:
auto = sc.parallelize(autos);

In [24]:
a1 = auto.filter(lambda x: x[5] == 'Bs As' and x[6][5] == '4')\
            .map(lambda x: ((x[4],x[1], x[2]), 1))\
            .reduceByKey(lambda x,y: x+y)
a1.collect()

[(('camion', 'marca5', 'modelo5'), 1),
 (('moto', 'marca11', 'modelo11'), 1),
 (('auto', 'marca1', 'modelo1'), 4),
 (('moto', 'marca7', 'modelo7'), 2),
 (('auto', 'marca9', 'modelo9'), 2)]

In [26]:
a2 = a1.map(lambda x: (x[0][0], (x[0][1], x[0][2], x[1])))\
    .reduceByKey(lambda x, y: (x) if x[2]>y[2] else (y))\
    .map(lambda x: (x[0], x[1][0], x[1][1]))
a2.collect()

[('auto', 'marca1', 'modelo1'),
 ('moto', 'marca7', 'modelo7'),
 ('camion', 'marca5', 'modelo5')]

2) Tenemos un dataframe con la información de distintas playlists
armadas por usuarios con el formato (playlist, song_id, description). A su vez, contamos con un dataframe de canciones que contiene (song_id, singer, year, lenght, genres).

Se pide generar un programa en Pandas que indique para cada playlist cual es el cantante predominante (con mas canciones incluidas dentro de esa lista). (**) (15 pts)

In [29]:
import pandas as pd

In [31]:
playlist = {'playlist': [11,11,11,14,15,
                         15,17,17,19,19,
                         17,17,19,11,15,
                         15],
           'song_id': [1,2,3,4,1,
                       6,2,8,9,1,
                       1,2,3,1,2,
                      3],
           'description': ['sd','sd','sd','sd','sd',
                           'sd','sd','sd','sd', 'sd',
                          'sd','sd','sd','sd','sd','sd']}

canciones = {'song_id': [1,2,3,4,5,6,7,8,9],
            'singer': ['s1', 's1', 's1', 's4', 's5', 's6', 's7', 's8', 's9'],
            'year': ['2019', '2018','2019', '2018','2019', '2018','2019', '2018','2019'],
            'lenght': [4,5,6,7,4,4,4,4,4]}

play = pd.DataFrame(playlist)
song = pd.DataFrame(canciones)

In [32]:
p1 = play[['playlist', 'song_id']].merge(song[['song_id', 'singer']], how='inner', on='song_id')

In [35]:
p1.head()

,playlist,song_id,singer
0,11,1,s1
1,15,1,s1
2,19,1,s1
3,17,1,s1
4,11,1,s1


In [112]:
p2 = p1.groupby(['playlist', 'singer']).agg({'song_id': 'count'})\
                    .sort_values('song_id').groupby(level=0).tail(1)\
                .reset_index()
p2.columns = ['playlist', 'singer', 'cant_singer_popular']
p2

,playlist,singer,cant_singer_popular
0,14,s4,1
1,19,s1,2
2,15,s1,3
3,17,s1,3
4,11,s1,4
